In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import time
from time import time, sleep
import pprint
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import re
from datetime import datetime
from selenium.common.exceptions import ElementNotVisibleException, StaleElementReferenceException, NoSuchElementException

In [6]:
df = pd.read_csv("bbc_selenium_scrape.csv", )
df.drop(columns = 'Unnamed: 0', inplace = True)
df

,title,date,competition,text,url
0,Wolves 0-1 Man Utd: Marcus Rashford comes off ...,SAT 31 Dec 2022,premier-league,Marcus Rashford turned from villain to hero as...,https://www.bbc.co.uk/sport/football/64083698
1,Bournemouth 0-2 Crystal Palace: Gary O'Neil la...,SAT 31 Dec 2022,premier-league,Bournemouth have conceded more first-half goal...,https://www.bbc.co.uk/sport/football/64083697
2,Fulham 2-1 Southampton: Saints stay bottom aft...,SAT 31 Dec 2022,premier-league,Palhinha's winner is his third Fulham goal sin...,https://www.bbc.co.uk/sport/football/64083700
3,Manchester City 1-1 Everton: Demarai Gray canc...,SAT 31 Dec 2022,premier-league,Haaland's goal was his 21st of the Premier Lea...,https://www.bbc.co.uk/sport/football/64083701
4,Newcastle United 0-0 Leeds United: Magpies hel...,SAT 31 Dec 2022,premier-league,Chris Wood was denied a goal against his forme...,https://www.bbc.co.uk/sport/football/64083702
...,...,...,...,...,...
1691,Sweden 1-0 Belgium: Linda Sembrant hits dramat...,FRI 22 Jul 2022,womens-european-championship,"Sweden will need an ""extremely good plan"" when...",https://www.bbc.co.uk/sport/football/62241465
1692,France 1-0 Netherlands: Eve Perisset scores ex...,SAT 23 Jul 2022,womens-european-championship,"France boss Corinne Diacre says her side ""are ...",https://www.bbc.co.uk/sport/football/62274347
1693,England 4-0 Sweden: Lionesses reach Euro 2022 ...,TUE 26 Jul 2022,womens-european-championship,England reached their first major women's tour...,https://www.bbc.co.uk/sport/football/62286243
1694,Germany 2-1 France: England to face eight-time...,WED 27 Jul 2022,womens-european-championship,Eight-time European champions Germany will pla...,https://www.bbc.co.uk/sport/football/62301014


In [7]:
df.drop_duplicates(inplace = True)
df

,title,date,competition,text,url
0,Wolves 0-1 Man Utd: Marcus Rashford comes off ...,SAT 31 Dec 2022,premier-league,Marcus Rashford turned from villain to hero as...,https://www.bbc.co.uk/sport/football/64083698
1,Bournemouth 0-2 Crystal Palace: Gary O'Neil la...,SAT 31 Dec 2022,premier-league,Bournemouth have conceded more first-half goal...,https://www.bbc.co.uk/sport/football/64083697
2,Fulham 2-1 Southampton: Saints stay bottom aft...,SAT 31 Dec 2022,premier-league,Palhinha's winner is his third Fulham goal sin...,https://www.bbc.co.uk/sport/football/64083700
3,Manchester City 1-1 Everton: Demarai Gray canc...,SAT 31 Dec 2022,premier-league,Haaland's goal was his 21st of the Premier Lea...,https://www.bbc.co.uk/sport/football/64083701
4,Newcastle United 0-0 Leeds United: Magpies hel...,SAT 31 Dec 2022,premier-league,Chris Wood was denied a goal against his forme...,https://www.bbc.co.uk/sport/football/64083702
...,...,...,...,...,...
1691,Sweden 1-0 Belgium: Linda Sembrant hits dramat...,FRI 22 Jul 2022,womens-european-championship,"Sweden will need an ""extremely good plan"" when...",https://www.bbc.co.uk/sport/football/62241465
1692,France 1-0 Netherlands: Eve Perisset scores ex...,SAT 23 Jul 2022,womens-european-championship,"France boss Corinne Diacre says her side ""are ...",https://www.bbc.co.uk/sport/football/62274347
1693,England 4-0 Sweden: Lionesses reach Euro 2022 ...,TUE 26 Jul 2022,womens-european-championship,England reached their first major women's tour...,https://www.bbc.co.uk/sport/football/62286243
1694,Germany 2-1 France: England to face eight-time...,WED 27 Jul 2022,womens-european-championship,Eight-time European champions Germany will pla...,https://www.bbc.co.uk/sport/football/62301014


In [8]:
df.dtypes

title          object
date           object
competition    object
text           object
url            object
dtype: object

In [9]:
march = pd.read_csv("bbc_selenium_march.csv", )
march.drop(columns = 'Unnamed: 0', inplace = True)
march

,title,date,competition,text,url
0,Arsenal 4-1 Crystal Palace: Gunners go eight p...,SUN 19 Mar 2023,premier-league,Gabriel Martinelli also scored the first goal ...,https://www.bbc.co.uk/sport/football/64929886
1,Aston Villa 3-0 Bournemouth: Unai Emery's side...,SAT 18 Mar 2023,premier-league,Jacob Ramsey scored Aston Villa's second as th...,https://www.bbc.co.uk/sport/football/64923720
2,Brentford 1-1 Leicester City: Foxes end Premie...,SAT 18 Mar 2023,premier-league,Harvey Barnes' goal was his ninth of the seaso...,https://www.bbc.co.uk/sport/football/64923721
3,Southampton 3-3 Tottenham: Saints come from be...,SAT 18 Mar 2023,premier-league,Ward-Prowse has scored seven Premier League go...,https://www.bbc.co.uk/sport/football/64923719
4,Wolverhampton Wanderers 2-4 Leeds United: Rare...,SAT 18 Mar 2023,premier-league,Rasmus Kristensen scored Leeds' third moments ...,https://www.bbc.co.uk/sport/football/64923722
...,...,...,...,...,...
94,Roma 0-1 Barcelona: Teenager Salma Paralluelo ...,TUE 21 Mar 2023,womens-champions-league,Barcelona have reached three Women's Champions...,https://www.bbc.co.uk/sport/football/64959155
95,Lewes 1-3 Manchester United: Red Devils reach ...,SUN 19 Mar 2023,fa-womens-cup,FA Cup: Man Utd beat Lewes 3-1 to secure place...,https://www.bbc.co.uk/sport/football/65006949
96,Birmingham City 0-2 Brighton: Seagulls into FA...,SUN 19 Mar 2023,fa-womens-cup,Danielle Carter is Brighton's highest goalscor...,https://www.bbc.co.uk/sport/football/65006956
97,Reading 1-3 Chelsea: Guro Reiten stars as Blue...,SUN 19 Mar 2023,fa-womens-cup,Chelsea have reached the Women's FA Cup semi-f...,https://www.bbc.co.uk/sport/football/65006963


In [11]:
df = pd.concat([df,march], ignore_index = True)
df

,title,date,competition,text,url
0,Wolves 0-1 Man Utd: Marcus Rashford comes off ...,SAT 31 Dec 2022,premier-league,Marcus Rashford turned from villain to hero as...,https://www.bbc.co.uk/sport/football/64083698
1,Bournemouth 0-2 Crystal Palace: Gary O'Neil la...,SAT 31 Dec 2022,premier-league,Bournemouth have conceded more first-half goal...,https://www.bbc.co.uk/sport/football/64083697
2,Fulham 2-1 Southampton: Saints stay bottom aft...,SAT 31 Dec 2022,premier-league,Palhinha's winner is his third Fulham goal sin...,https://www.bbc.co.uk/sport/football/64083700
3,Manchester City 1-1 Everton: Demarai Gray canc...,SAT 31 Dec 2022,premier-league,Haaland's goal was his 21st of the Premier Lea...,https://www.bbc.co.uk/sport/football/64083701
4,Newcastle United 0-0 Leeds United: Magpies hel...,SAT 31 Dec 2022,premier-league,Chris Wood was denied a goal against his forme...,https://www.bbc.co.uk/sport/football/64083702
...,...,...,...,...,...
1200,Roma 0-1 Barcelona: Teenager Salma Paralluelo ...,TUE 21 Mar 2023,womens-champions-league,Barcelona have reached three Women's Champions...,https://www.bbc.co.uk/sport/football/64959155
1201,Lewes 1-3 Manchester United: Red Devils reach ...,SUN 19 Mar 2023,fa-womens-cup,FA Cup: Man Utd beat Lewes 3-1 to secure place...,https://www.bbc.co.uk/sport/football/65006949
1202,Birmingham City 0-2 Brighton: Seagulls into FA...,SUN 19 Mar 2023,fa-womens-cup,Danielle Carter is Brighton's highest goalscor...,https://www.bbc.co.uk/sport/football/65006956
1203,Reading 1-3 Chelsea: Guro Reiten stars as Blue...,SUN 19 Mar 2023,fa-womens-cup,Chelsea have reached the Women's FA Cup semi-f...,https://www.bbc.co.uk/sport/football/65006963


In [12]:
df.drop_duplicates(inplace = True)
df

,title,date,competition,text,url
0,Wolves 0-1 Man Utd: Marcus Rashford comes off ...,SAT 31 Dec 2022,premier-league,Marcus Rashford turned from villain to hero as...,https://www.bbc.co.uk/sport/football/64083698
1,Bournemouth 0-2 Crystal Palace: Gary O'Neil la...,SAT 31 Dec 2022,premier-league,Bournemouth have conceded more first-half goal...,https://www.bbc.co.uk/sport/football/64083697
2,Fulham 2-1 Southampton: Saints stay bottom aft...,SAT 31 Dec 2022,premier-league,Palhinha's winner is his third Fulham goal sin...,https://www.bbc.co.uk/sport/football/64083700
3,Manchester City 1-1 Everton: Demarai Gray canc...,SAT 31 Dec 2022,premier-league,Haaland's goal was his 21st of the Premier Lea...,https://www.bbc.co.uk/sport/football/64083701
4,Newcastle United 0-0 Leeds United: Magpies hel...,SAT 31 Dec 2022,premier-league,Chris Wood was denied a goal against his forme...,https://www.bbc.co.uk/sport/football/64083702
...,...,...,...,...,...
1196,Arsenal 2-0 Bayern (2-1 agg): Arsenal stun Bay...,WED 29 Mar 2023,womens-champions-league,Arsenal remain the only English side to win th...,https://www.bbc.co.uk/sport/football/65098325
1197,Lyon 0-1 Chelsea: Blues get crucial European w...,WED 22 Mar 2023,womens-champions-league,Guro Reiten scored with Chelsea's first shot o...,https://www.bbc.co.uk/sport/football/65029776
1198,Paris St-Germain 0-1 Wolfsburg: Penalty gives ...,WED 22 Mar 2023,womens-champions-league,Wolfsburg have not lost a match in the Women's...,https://www.bbc.co.uk/sport/football/65046341
1199,Bayern Munich 1-0 Arsenal: Gunners trail Champ...,TUE 21 Mar 2023,womens-champions-league,Bayern Munich's Lea Schuller scored her 14th g...,https://www.bbc.co.uk/sport/football/64959148


In [19]:
womens_comps = ["womens-super-league", "womens-champions-league", "fa-womens-cup",
"womens-european-championships-qualifying",
"womens-world-cup-qualifying-european",
"womens-international-friendlies", "shebelieves-cup", "arnold-clark-cup", "womens-european-championship", "womens-world-cup"]

womens = [1 if x in womens_comps else 0 for x in df.competition]

df["womens"] = womens

df.head()

,title,date,competition,text,url,womens
0,Wolves 0-1 Man Utd: Marcus Rashford comes off ...,SAT 31 Dec 2022,premier-league,Marcus Rashford turned from villain to hero as...,https://www.bbc.co.uk/sport/football/64083698,0
1,Bournemouth 0-2 Crystal Palace: Gary O'Neil la...,SAT 31 Dec 2022,premier-league,Bournemouth have conceded more first-half goal...,https://www.bbc.co.uk/sport/football/64083697,0
2,Fulham 2-1 Southampton: Saints stay bottom aft...,SAT 31 Dec 2022,premier-league,Palhinha's winner is his third Fulham goal sin...,https://www.bbc.co.uk/sport/football/64083700,0
3,Manchester City 1-1 Everton: Demarai Gray canc...,SAT 31 Dec 2022,premier-league,Haaland's goal was his 21st of the Premier Lea...,https://www.bbc.co.uk/sport/football/64083701,0
4,Newcastle United 0-0 Leeds United: Magpies hel...,SAT 31 Dec 2022,premier-league,Chris Wood was denied a goal against his forme...,https://www.bbc.co.uk/sport/football/64083702,0


In [21]:
international_comps = ["european-championship-qualifying",  "international-friendlies",
"womens-european-championships-qualifying", "womens-world-cup-qualifying-european",
"womens-international-friendlies", "shebelieves-cup", "arnold-clark-cup", "world-cup", "european-championship", 
"world-cup", "womens-european-championship", "womens-world-cup"]

internationals = [1 if x in international_comps else 0 for x in df.competition]

df["international"] = internationals

domestic_comps = ["premier-league", "fa-cup", "womens-super-league",  "fa-womens-cup",
"womens-european-championships-qualifying"]

domestic = [1 if x in domestic_comps else 0 for x in df.competition]

df["domestic"] = domestic

df.head()

,title,date,competition,text,url,womens,international,domestic
0,Wolves 0-1 Man Utd: Marcus Rashford comes off ...,SAT 31 Dec 2022,premier-league,Marcus Rashford turned from villain to hero as...,https://www.bbc.co.uk/sport/football/64083698,0,0,1
1,Bournemouth 0-2 Crystal Palace: Gary O'Neil la...,SAT 31 Dec 2022,premier-league,Bournemouth have conceded more first-half goal...,https://www.bbc.co.uk/sport/football/64083697,0,0,1
2,Fulham 2-1 Southampton: Saints stay bottom aft...,SAT 31 Dec 2022,premier-league,Palhinha's winner is his third Fulham goal sin...,https://www.bbc.co.uk/sport/football/64083700,0,0,1
3,Manchester City 1-1 Everton: Demarai Gray canc...,SAT 31 Dec 2022,premier-league,Haaland's goal was his 21st of the Premier Lea...,https://www.bbc.co.uk/sport/football/64083701,0,0,1
4,Newcastle United 0-0 Leeds United: Magpies hel...,SAT 31 Dec 2022,premier-league,Chris Wood was denied a goal against his forme...,https://www.bbc.co.uk/sport/football/64083702,0,0,1


In [36]:
df['length'] = df.text.str.len()
df.head()

,title,date,competition,text,url,womens,international,domestic,length
0,Wolves 0-1 Man Utd: Marcus Rashford comes off ...,SAT 31 Dec 2022,premier-league,Marcus Rashford turned from villain to hero as...,https://www.bbc.co.uk/sport/football/64083698,0,0,1,4254.0
1,Bournemouth 0-2 Crystal Palace: Gary O'Neil la...,SAT 31 Dec 2022,premier-league,Bournemouth have conceded more first-half goal...,https://www.bbc.co.uk/sport/football/64083697,0,0,1,4879.0
2,Fulham 2-1 Southampton: Saints stay bottom aft...,SAT 31 Dec 2022,premier-league,Palhinha's winner is his third Fulham goal sin...,https://www.bbc.co.uk/sport/football/64083700,0,0,1,3867.0
3,Manchester City 1-1 Everton: Demarai Gray canc...,SAT 31 Dec 2022,premier-league,Haaland's goal was his 21st of the Premier Lea...,https://www.bbc.co.uk/sport/football/64083701,0,0,1,5170.0
4,Newcastle United 0-0 Leeds United: Magpies hel...,SAT 31 Dec 2022,premier-league,Chris Wood was denied a goal against his forme...,https://www.bbc.co.uk/sport/football/64083702,0,0,1,3590.0


In [39]:
df.womens.value_counts(normalize = True)

0    0.730396
1    0.269604
Name: womens, dtype: float64

In [38]:
df.groupby("womens")['length'].mean()

womens
0    3602.098063
1    3021.074349
Name: length, dtype: float64

In [40]:
df.groupby("international")['length'].mean()

international
0    3426.946324
1    3576.088235
Name: length, dtype: float64

In [41]:
df.groupby("domestic")['length'].mean()

domestic
0    3354.282776
1    3517.260623
Name: length, dtype: float64

In [42]:
womens = df[df.womens == 1]
womens

,title,date,competition,text,url,womens,international,domestic,length
499,Tottenham 0-3 Everton: Toffees end losing run ...,WED 14 Dec 2022,womens-super-league,WSL 2022: Everton stroll to 3-0 win over Totte...,https://www.bbc.co.uk/sport/football/63443687,1,0,1,2347.0
500,Manchester City 1-1 Manchester United: Enterta...,SUN 11 Dec 2022,womens-super-league,Man City held by Man Utd in thrilling derbyMan...,https://www.bbc.co.uk/sport/football/63849445,1,0,1,4157.0
501,Tottenham 0-2 West Ham: Dagny Brynjarsdottir a...,SUN 11 Dec 2022,womens-super-league,Dagny Brynjarsdottir has scored five WSL goals...,https://www.bbc.co.uk/sport/football/63849444,1,0,1,2084.0
502,Aston Villa 1-4 Arsenal: Gunners come from beh...,SUN 11 Dec 2022,womens-super-league,WSL 2022: Hanson scores screamer as Villa lose...,https://www.bbc.co.uk/sport/football/63849447,1,0,1,4477.0
503,Chelsea 3-2 Reading: Leaders survive scare to ...,SUN 11 Dec 2022,womens-super-league,Fran Kirby slotted in to put Chelsea ahead as ...,https://www.bbc.co.uk/sport/football/63849450,1,0,1,3338.0
...,...,...,...,...,...,...,...,...,...
1196,Arsenal 2-0 Bayern (2-1 agg): Arsenal stun Bay...,WED 29 Mar 2023,womens-champions-league,Arsenal remain the only English side to win th...,https://www.bbc.co.uk/sport/football/65098325,1,0,0,3263.0
1197,Lyon 0-1 Chelsea: Blues get crucial European w...,WED 22 Mar 2023,womens-champions-league,Guro Reiten scored with Chelsea's first shot o...,https://www.bbc.co.uk/sport/football/65029776,1,0,0,3917.0
1198,Paris St-Germain 0-1 Wolfsburg: Penalty gives ...,WED 22 Mar 2023,womens-champions-league,Wolfsburg have not lost a match in the Women's...,https://www.bbc.co.uk/sport/football/65046341,1,0,0,1531.0
1199,Bayern Munich 1-0 Arsenal: Gunners trail Champ...,TUE 21 Mar 2023,womens-champions-league,Bayern Munich's Lea Schuller scored her 14th g...,https://www.bbc.co.uk/sport/football/64959148,1,0,0,3818.0


In [43]:
womens.groupby("international")["length"].mean()

international
0    2724.005128
1    3803.891892
Name: length, dtype: float64

In [44]:
import matplotlib.pyplot as plt
import seaborn as sns